# 土炮股票分析系統

<img src="system.png" />

## 抓取公司資料

In [1]:
import pandas as pd

In [2]:
url = "http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange=NASDAQ&render=download"
data = pd.read_csv(url)

In [3]:
data.head()

,Symbol,Name,LastSale,MarketCap,ADR TSO,IPOyear,Sector,Industry,Summary Quote,Unnamed: 9
0,PIH,"1347 Property Insurance Holdings, Inc.",7.6,4.527142e+07,n/a,2014,Finance,Property-Casualty Insurers,http://www.nasdaq.com/symbol/pih,NaN
1,FLWS,"1-800 FLOWERS.COM, Inc.",10.3,6.753613e+08,n/a,1999,Consumer Services,Other Specialty Stores,http://www.nasdaq.com/symbol/flws,NaN
2,FCCY,1st Constitution Bancorp (NJ),18.1,1.452949e+08,n/a,n/a,Finance,Savings Institutions,http://www.nasdaq.com/symbol/fccy,NaN
3,SRCE,1st Source Corporation,45.6,1.226308e+09,n/a,n/a,Finance,Major Banks,http://www.nasdaq.com/symbol/srce,NaN
4,VNET,"21Vianet Group, Inc.",5.71,3.309771e+08,57964466,2011,Technology,"Computer Software: Programming, Data Processing",http://www.nasdaq.com/symbol/vnet,NaN


In [6]:
companylist = data['Symbol'][0:10].tolist()

## 波動率選股

In [7]:
import pandas_datareader.data as web

from datetime import datetime
import numpy as np

In [9]:
def calculate(symbol):
    df=web.DataReader(symbol, 'yahoo', datetime(2016,1,1))
    dailyRet = df['Close'].pct_change()
    return dailyRet.std() * np.sqrt(dailyRet.count())

In [10]:
results = []
for symbol in companylist:
    vo = calculate(symbol)
    results.append((vo, symbol))
results.sort()
results

[(0.25783714378670841, 'SRCE'),
 (0.3124805208772693, 'FCCY'),
 (0.33096186965160107, 'JOBS'),
 (0.34097241273844719, 'CAFD'),
 (0.39445385991868348, 'AVHI'),
 (0.40187274471660905, 'PIH'),
 (0.42943106538227327, 'FLWS'),
 (0.43279924882919563, 'TWOU'),
 (0.43610691109647881, 'EGHT'),
 (0.61649467489464382, 'VNET')]

In [15]:
computer_selected = np.array(results)[:, 1][:5].tolist()
computer_selected

['SRCE', 'FCCY', 'JOBS', 'CAFD', 'AVHI']

# 加入自選股

In [16]:
self_selected = ['TSLA', 'GOOG', 'YHOO', 'MSFT', 'AAPL']

In [18]:
candidates = computer_selected + self_selected

In [19]:
candidates = set(candidates)

In [20]:
candidates = list(candidates)

# 回測

In [52]:
# maxDD: maximum drawdown
# maxDDD: maximum drawdown duration

def calculateMaxDD(cumRet):
    highwatermark = np.zeros(np.size(cumRet))
    drawdownduration = np.zeros(np.size(cumRet))
    drawdown = np.zeros(np.size(cumRet))
    for t in range(2, cumRet.size):
        highwatermark[t] = max(highwatermark[t-1], cumRet[t])
        drawdown[t] = (1 + highwatermark[t]) / (1 + cumRet[t]) - 1
        if (drawdown[t] == 0):
            drawdownduration[t] = 0
        else:
            # 從日期來計算 MaxDDD 的天數
            drawdownduration[t] = drawdownduration[t-1] + (cumRet.index[t]-cumRet.index[t-1]).days
            #drawdownduration[t] = drawdownduration[t-1] + 1
    maxDD = max(drawdown)
    maxDDD = max(drawdownduration)
    #Series(drawdownduration, index=cumRet.index).plot()
    return maxDD, maxDDD

In [49]:
def indicators(df):
    dailyRet = df['Close'].pct_change()
    #假設無風險利率為 4%
    #假設一年有252個交易日
    excessRet = (dailyRet - 0.04/252)[df['positions']==1.0]
    SharpeRatio = np.sqrt(252.0)*np.mean(excessRet)/np.std(excessRet)
    
    cumRet = np.cumprod(1+excessRet) - 1
    
    maxdd, maxddd = calculateMaxDD(cumRet)
    
    return SharpeRatio, maxdd, maxddd

In [50]:
def strategy(df):
    # Donchian Channel
    df['20d_high'] = np.round(pd.Series.rolling(df['Close'], window=20).max(), 2)
    df['10d_low'] = np.round(pd.Series.rolling(df['Close'], window=10).min(), 2)

    has_position = False
    df['signals'] = np.zeros(df['Close'].shape)
    for t in range(2, df['signals'].size):
        if df['Close'][t] > df['20d_high'][t-1]:
            if not has_position:
                df.loc[df.index[t], 'signals'] = 1.0
                has_position = True
        elif df['Close'][t] < df['10d_low'][t-1]:
            if has_position:
                df.loc[df.index[t], 'signals'] = -1.0
                has_position = False

    df['positions'] = df['signals'].cumsum()
    return df

In [24]:
all_data = {}

for symbol in candidates:
    all_data[symbol] = web.DataReader(symbol, 'yahoo', datetime(2016,1,1))

In [53]:
results = []
for symbol in candidates:
    strategy(all_data[symbol])
    SharpeRatio, maxdd, maxddd = indicators(all_data[symbol])
    results.append((SharpeRatio, maxdd, maxddd, symbol))

results

[(4.7437238093013292, 0.028184773453298506, 41.0, 'AAPL'),
 (3.1906377133025217, 0.061363744885344795, 140.0, 'YHOO'),
 (3.0951853865829002, 0.063731057069693176, 80.0, 'SRCE'),
 (5.2871348417034438, 0.07171142504927519, 77.0, 'TSLA'),
 (1.6168150645512951, 0.062648284422384126, 88.0, 'GOOG'),
 (3.2011008420921683, 0.070477704348795189, 145.0, 'FCCY'),
 (3.0323807731038852, 0.042810886085439037, 60.0, 'MSFT'),
 (2.0307236938234143, 0.093937685286554329, 127.0, 'JOBS'),
 (3.3518546962135547, 0.080871012332741632, 77.0, 'AVHI'),
 (2.7822634509541655, 0.098059598858270203, 93.0, 'CAFD')]

In [57]:
sorted(results, reverse=True)

[(5.2871348417034438, 0.07171142504927519, 77.0, 'TSLA'),
 (4.7437238093013292, 0.028184773453298506, 41.0, 'AAPL'),
 (3.3518546962135547, 0.080871012332741632, 77.0, 'AVHI'),
 (3.2011008420921683, 0.070477704348795189, 145.0, 'FCCY'),
 (3.1906377133025217, 0.061363744885344795, 140.0, 'YHOO'),
 (3.0951853865829002, 0.063731057069693176, 80.0, 'SRCE'),
 (3.0323807731038852, 0.042810886085439037, 60.0, 'MSFT'),
 (2.7822634509541655, 0.098059598858270203, 93.0, 'CAFD'),
 (2.0307236938234143, 0.093937685286554329, 127.0, 'JOBS'),
 (1.6168150645512951, 0.062648284422384126, 88.0, 'GOOG')]